In [0]:
!pip install ConfigParser

In [1]:
!ls '/content/drive/My Drive/Colab Notebooks/twitter.properties'

'/content/drive/My Drive/Colab Notebooks/twitter.properties'


In [2]:
import configparser
config = configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/twitter.properties')
print(config.sections())

['twitter']


In [0]:
import os
import tweepy as tw

In [0]:
accesstoken= config.get('twitter','accesstoken')
accesstokensecret= config.get('twitter','accesstokensecret')
apikey= config.get('twitter','apikey')
apisecretkey= config.get('twitter','apisecretkey')

In [0]:
auth = tw.OAuthHandler(apikey,apisecretkey)
auth.set_access_token(accesstoken,accesstokensecret)
api = tw.API(auth, wait_on_rate_limit=True)

In [0]:
search_words="#coronavirus"
date_since = "2020-01-22"

In [0]:
tweets = tw.Cursor(api.search,
                   q=search_words,
                   lang="en",
                   since=date_since).items(500)

In [17]:
tweets

In [18]:
tweet_details = [[tweet.geo,tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]
tweet_details

[[None,
  "#Newsoftheday - Discover today's front pages of the #BelgianPress \U0001f9d0💡📰!\n#Tourism #hazard #kompany #coronavirus… https://t.co/roJx01qvWy",
  'Auxipress',
  'Belgium'],
 [None,
  'RT @parvinfarazmand: Iran: #Coronavirus Update, Over 44,800 Deaths, May 27, 2020, 6:00 PM CEST - NCRI https://t.co/TSKRvpO31G #HumanRightsV…',
  'davedaram',
  ''],
 [None,
  "RT @ashoswai: #Coronavirus has not stopped Modi-Shah's police to target Muslim students for participating in anti-citizenship act protest!!…",
  'israrul8',
  ''],
 [None,
  'RT @Autrice: Does anyone still read the #NYT ? 🤣🤣\n\n“The New York Times admitted this week the doctor didn’t die and didn’t have coronavirus…',
  'Aliciastarr001',
  'California (Bay Area)'],
 [None,
  'RT @getpostman: Those on the front lines of the COVID-19 fight need quick, easy access to real-time critical data. That’s what APIs (and AP…',
  'sounakpal',
  'India'],
 [None,
  '"Freedom will become the rule again and prohibition will be the ex

In [0]:
import pandas as pd

In [0]:
tweet_df = pd.DataFrame(data=tweet_details, columns=['geo','text','user','location'])

In [21]:
pd.set_option('max_colwidth',800)
tweet_df

,geo,text,user,location
0,None,#Newsoftheday - Discover today's front pages of the #BelgianPress 🧐💡📰!\n#Tourism #hazard #kompany #coronavirus… https://t.co/roJx01qvWy,Auxipress,Belgium
1,None,"RT @parvinfarazmand: Iran: #Coronavirus Update, Over 44,800 Deaths, May 27, 2020, 6:00 PM CEST - NCRI https://t.co/TSKRvpO31G #HumanRightsV…",davedaram,
2,None,RT @ashoswai: #Coronavirus has not stopped Modi-Shah's police to target Muslim students for participating in anti-citizenship act protest!!…,israrul8,
3,None,RT @Autrice: Does anyone still read the #NYT ? 🤣🤣\n\n“The New York Times admitted this week the doctor didn’t die and didn’t have coronavirus…,Aliciastarr001,California (Bay Area)
4,None,"RT @getpostman: Those on the front lines of the COVID-19 fight need quick, easy access to real-time critical data. That’s what APIs (and AP…",sounakpal,India
...,...,...,...,...
495,None,TRUMP CAN'T HIDE THE TRUTH FOREVER. IT WILL COME OUT VERY SOON. WHEN THE #CORONAVIRUS NUMBERS OF CASES &amp; DEATHS SPI… https://t.co/3sbh3XTU0Y,TomthunkitsMind,"Atlanta, GA"
496,None,This doesn’t seem to suggest that lifting lockdown is going to keep R and infections down. Sad times #coronavirus… https://t.co/QhFeaq5NPw,physics_jamie,Kent
497,None,RT @Anna_Soubry: From Monday? Have you seen the weather forecast? And there’s the small problem that most people don’t give a stuff about…,alexcburrows,Birmingham
498,None,"RT @LGAcomms: Vital emergency Govt funding has helped, but #LocalGov could need as much as £6bn more to cover the financial impact of #coro…",StratfordDC,"Stratford-upon-Avon, England"


In [22]:
tweet_df.user.value_counts()

MichelleRoseG33    5
NSMCcomLTD1        3
TheKillidGroup     2
GlobalNewsBKK      2
corona_china       2
                  ..
PierreGrosser      1
StapleTye          1
the_arabposts      1
saimasyed01        1
Coopersphere       1
Name: user, Length: 476, dtype: int64

In [23]:
tweet_df.location.value_counts()

                             136
London, England                9
India                          8
UK                             5
Los Angeles, CA                5
                            ... 
Delhi NCR/Lucknow/Chennai      1
Roseville, CA                  1
Minnesota, USA                 1
مكة المكرمة                    1
Bodfish, CA                    1
Name: location, Length: 291, dtype: int64

In [0]:
import re

def clean_tweets(text):
  text = re.sub("RT @[\w]*:","",text) #RT followed by @ followed by 0 or more combintion of words followed by :
  text = re.sub("@[\w]*","",text)  # @ followed by 0 or more combintion of words followed by :
  text = re.sub("https?://[A-Za-z0-9./]*","",text) # https or http followed by 0 or more combination of any alphabets or digits
  text = re.sub("\n","",text)
  return text

In [0]:
tweet_df['text'] = tweet_df['text'].apply(lambda x: clean_tweets(x))

In [26]:
tweet_df['text']

0                       #Newsoftheday - Discover today's front pages of the #BelgianPress 🧐💡📰!#Tourism #hazard #kompany #coronavirus… 
1                                     Iran: #Coronavirus Update, Over 44,800 Deaths, May 27, 2020, 6:00 PM CEST - NCRI  #HumanRightsV…
2       #Coronavirus has not stopped Modi-Shah's police to target Muslim students for participating in anti-citizenship act protest!!…
3        Does anyone still read the #NYT ? 🤣🤣“The New York Times admitted this week the doctor didn’t die and didn’t have coronavirus…
4         Those on the front lines of the COVID-19 fight need quick, easy access to real-time critical data. That’s what APIs (and AP…
                                                                    ...                                                               
495          TRUMP CAN'T HIDE THE TRUTH FOREVER. IT WILL COME OUT VERY SOON. WHEN THE #CORONAVIRUS NUMBERS OF CASES &amp; DEATHS SPI… 
496                This doesn’t seem to suggest that li

In [27]:
tweet_df.head(20)

,geo,text,user,location
0,None,#Newsoftheday - Discover today's front pages of the #BelgianPress 🧐💡📰!#Tourism #hazard #kompany #coronavirus…,Auxipress,Belgium
1,None,"Iran: #Coronavirus Update, Over 44,800 Deaths, May 27, 2020, 6:00 PM CEST - NCRI #HumanRightsV…",davedaram,
2,None,#Coronavirus has not stopped Modi-Shah's police to target Muslim students for participating in anti-citizenship act protest!!…,israrul8,
3,None,Does anyone still read the #NYT ? 🤣🤣“The New York Times admitted this week the doctor didn’t die and didn’t have coronavirus…,Aliciastarr001,California (Bay Area)
4,None,"Those on the front lines of the COVID-19 fight need quick, easy access to real-time critical data. That’s what APIs (and AP…",sounakpal,India
5,None,"""Freedom will become the rule again and prohibition will be the exception,"" «The end of #confinement will allow us…",domwabs,"Lyon, France"
6,None,Two countries united efforts to kill the #CoronaVirus in between them.PS. Sharp weapons used,Vas_Ah,"Jammu, Jammu And Kashmir"
7,None,Keeping the spirit of generosity alive during #COVID19 pandemic. #relief programme at Thaiba Garden and near by villages.…,rafeek_rapi,
8,None,"""Government rejected radical lockdown of England's care homes"" #coronavirus Is anyone surprised? This govt is so cl…",buddah_uk,Not in Surrey
9,None,The smoking gun of govt care home policy?Govt advice from 2 April.“Some of these patients may have #COVOD19. All ca…,tonyevansace,


In [0]:
tweet_df.to_csv('tweets.csv')

In [0]:
!ls

drive  sample_data  tweets.csv


In [0]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [30]:
tweet_df['text'].apply(lambda x: [print("\tText: {}, Entity: {}".format(ent.text, ent.label_)) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

	Text: today, Entity: DATE
	Text: Tourism, Entity: PERSON
	Text: Iran, Entity: GPE
	Text: Over 44,800, Entity: CARDINAL
	Text: May 27, 2020, Entity: DATE
	Text: 6:00 PM, Entity: TIME
	Text: CEST, Entity: PERSON
	Text: HumanRightsV, Entity: PERSON
	Text: Coronavirus, Entity: PERSON
	Text: Modi-Shah's, Entity: ORG
	Text: Muslim, Entity: NORP
	Text: 🤣🤣, Entity: PERSON
	Text: The New York Times, Entity: ORG
	Text: this week, Entity: DATE
	Text: AP, Entity: ORG
	Text: Two, Entity: CARDINAL
	Text: CoronaVirus, Entity: PERSON
	Text: Thaiba Garden, Entity: FAC
	Text: England, Entity: GPE
	Text: 2 April, Entity: DATE
	Text: COVOD19, Entity: MONEY
	Text: Douche, Entity: ORG
	Text: one day, Entity: DATE
	Text: ExecutiveOrder, Entity: MONEY
	Text: Coronavirus, Entity: PERSON
	Text: Modi-Shah's, Entity: ORG
	Text: Muslim, Entity: NORP
	Text: Vital, Entity: ORG
	Text: Govt, Entity: ORG
	Text: LocalGov, Entity: PERSON
	Text: as much as £6bn, Entity: MONEY
	Text: Thursday, Entity: DATE
	Text: almost 3

0                            [None, None, ]
1      [None, None, None, None, None, None]
2                        [None, None, None]
3                        [None, None, None]
4                                    [None]
                       ...                 
495                                      []
496                                      []
497                                  [None]
498                [None, None, None, None]
499                [None, None, None, None]
Name: text, Length: 500, dtype: object

In [0]:
tweet_df['entities']=tweet_df['text'].apply(lambda x: [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

In [0]:
# tweet_df.drop(['entities'],axis=1,inplace=True)

In [32]:
tweet_df.head(20)

,geo,text,user,location,entities
0,None,#Newsoftheday - Discover today's front pages of the #BelgianPress 🧐💡📰!#Tourism #hazard #kompany #coronavirus…,Auxipress,Belgium,"[(today, DATE), (Tourism, PERSON), ]"
1,None,"Iran: #Coronavirus Update, Over 44,800 Deaths, May 27, 2020, 6:00 PM CEST - NCRI #HumanRightsV…",davedaram,,"[(Iran, GPE), (Over 44,800, CARDINAL), (May 27, 2020, DATE), (6:00 PM, TIME), (CEST, PERSON), (HumanRightsV, PERSON)]"
2,None,#Coronavirus has not stopped Modi-Shah's police to target Muslim students for participating in anti-citizenship act protest!!…,israrul8,,"[(Coronavirus, PERSON), (Modi-Shah's, ORG), (Muslim, NORP)]"
3,None,Does anyone still read the #NYT ? 🤣🤣“The New York Times admitted this week the doctor didn’t die and didn’t have coronavirus…,Aliciastarr001,California (Bay Area),"[(🤣🤣, PERSON), (The New York Times, ORG), (this week, DATE)]"
4,None,"Those on the front lines of the COVID-19 fight need quick, easy access to real-time critical data. That’s what APIs (and AP…",sounakpal,India,"[(AP, ORG)]"
5,None,"""Freedom will become the rule again and prohibition will be the exception,"" «The end of #confinement will allow us…",domwabs,"Lyon, France",[]
6,None,Two countries united efforts to kill the #CoronaVirus in between them.PS. Sharp weapons used,Vas_Ah,"Jammu, Jammu And Kashmir","[(Two, CARDINAL), (CoronaVirus, PERSON)]"
7,None,Keeping the spirit of generosity alive during #COVID19 pandemic. #relief programme at Thaiba Garden and near by villages.…,rafeek_rapi,,"[(Thaiba Garden, FAC)]"
8,None,"""Government rejected radical lockdown of England's care homes"" #coronavirus Is anyone surprised? This govt is so cl…",buddah_uk,Not in Surrey,"[(England, GPE)]"
9,None,The smoking gun of govt care home policy?Govt advice from 2 April.“Some of these patients may have #COVOD19. All ca…,tonyevansace,,"[(2 April, DATE), (COVOD19, MONEY)]"


In [33]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
sid = SentimentIntensityAnalyzer()

In [0]:
tweet_df['sentiment'] = tweet_df['text'].apply(lambda x: sid.polarity_scores(x))

In [36]:
tweet_df.head(10)

,geo,text,user,location,entities,sentiment
0,None,#Newsoftheday - Discover today's front pages of the #BelgianPress 🧐💡📰!#Tourism #hazard #kompany #coronavirus…,Auxipress,Belgium,"[(today, DATE), (Tourism, PERSON), ]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
1,None,"Iran: #Coronavirus Update, Over 44,800 Deaths, May 27, 2020, 6:00 PM CEST - NCRI #HumanRightsV…",davedaram,,"[(Iran, GPE), (Over 44,800, CARDINAL), (May 27, 2020, DATE), (6:00 PM, TIME), (CEST, PERSON), (HumanRightsV, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
2,None,#Coronavirus has not stopped Modi-Shah's police to target Muslim students for participating in anti-citizenship act protest!!…,israrul8,,"[(Coronavirus, PERSON), (Modi-Shah's, ORG), (Muslim, NORP)]","{'neg': 0.0, 'neu': 0.87, 'pos': 0.13, 'compound': 0.3071}"
3,None,Does anyone still read the #NYT ? 🤣🤣“The New York Times admitted this week the doctor didn’t die and didn’t have coronavirus…,Aliciastarr001,California (Bay Area),"[(🤣🤣, PERSON), (The New York Times, ORG), (this week, DATE)]","{'neg': 0.16, 'neu': 0.782, 'pos': 0.058, 'compound': -0.5423}"
4,None,"Those on the front lines of the COVID-19 fight need quick, easy access to real-time critical data. That’s what APIs (and AP…",sounakpal,India,"[(AP, ORG)]","{'neg': 0.183, 'neu': 0.709, 'pos': 0.108, 'compound': -0.25}"
5,None,"""Freedom will become the rule again and prohibition will be the exception,"" «The end of #confinement will allow us…",domwabs,"Lyon, France",[],"{'neg': 0.0, 'neu': 0.736, 'pos': 0.264, 'compound': 0.7269}"
6,None,Two countries united efforts to kill the #CoronaVirus in between them.PS. Sharp weapons used,Vas_Ah,"Jammu, Jammu And Kashmir","[(Two, CARDINAL), (CoronaVirus, PERSON)]","{'neg': 0.355, 'neu': 0.514, 'pos': 0.131, 'compound': -0.7003}"
7,None,Keeping the spirit of generosity alive during #COVID19 pandemic. #relief programme at Thaiba Garden and near by villages.…,rafeek_rapi,,"[(Thaiba Garden, FAC)]","{'neg': 0.0, 'neu': 0.664, 'pos': 0.336, 'compound': 0.765}"
8,None,"""Government rejected radical lockdown of England's care homes"" #coronavirus Is anyone surprised? This govt is so cl…",buddah_uk,Not in Surrey,"[(England, GPE)]","{'neg': 0.147, 'neu': 0.625, 'pos': 0.228, 'compound': 0.2023}"
9,None,The smoking gun of govt care home policy?Govt advice from 2 April.“Some of these patients may have #COVOD19. All ca…,tonyevansace,,"[(2 April, DATE), (COVOD19, MONEY)]","{'neg': 0.106, 'neu': 0.752, 'pos': 0.142, 'compound': 0.2023}"


In [0]:
import googlemaps

In [38]:
pip install -U googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.4.1-cp36-none-any.whl size=37819 sha256=763354f94fb0ba3bbaac735024a2ac9d3aec0b53974f6afe5806663797c8d070
  Stored in directory: /root/.cache/pip/wheels/5c/ec/b9/9d0c26b80d7dcf16496c82e8108c93186c6d4df097a41bba4f
Successfully built googlemaps


In [40]:
gmaps = googlemaps.Client(key= config.get('twitter','googleapikey'))
gmaps

In [41]:
geocode_result = gmaps.geocode(tweet_df['location'][0])
print(geocode_result)
print(geocode_result[0]['formatted_address'])
print(geocode_result[0]['formatted_address'].split(",")[-1].strip())

[{'access_points': [], 'address_components': [{'long_name': 'Belgium', 'short_name': 'BE', 'types': ['country', 'political']}], 'formatted_address': 'Belgium', 'geometry': {'bounds': {'northeast': {'lat': 51.5051449, 'lng': 6.408124099999999}, 'southwest': {'lat': 49.497013, 'lng': 2.5240999}}, 'location': {'lat': 50.503887, 'lng': 4.469936}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 51.5051449, 'lng': 6.408124099999999}, 'southwest': {'lat': 49.497013, 'lng': 2.5240999}}}, 'place_id': 'ChIJl5fz7WR9wUcR8g_mObTy60c', 'types': ['country', 'political']}]
Belgium
Belgium


In [0]:
def get_country(input):
  try:
    output = gmaps.geocode(input)[0]['formatted_address'].split(",")[-1].strip()
  except:
    output="Error" # for special characters like smileys in location column, return error
  return output

In [0]:
tweet_df["country"] = tweet_df["location"].apply(lambda x: "" if (not x.strip()) else get_country(x))  # if empty, then will get nothing else get country name from function

In [44]:
tweet_df.country.value_counts()

                                136
USA                              91
UK                               75
India                            58
Error                            27
South Africa                      9
Pakistan                          8
United Kingdom                    8
Spain                             7
Canada                            7
France                            5
Australia                         5
Belgium                           4
Japan                             4
Thailand                          3
Taiwan                            3
Ireland                           3
Italy                             3
United States                     3
Germany                           3
Qatar                             2
Brazil                            2
Afghanistan                       2
Netherlands                       2
Dubai - United Arab Emirates      2
Philippines                       2
Europe                            2
Gaza Strip                  

In [48]:
tweet_df.head(10)

,geo,text,user,location,entities,sentiment,country
0,None,#Newsoftheday - Discover today's front pages of the #BelgianPress 🧐💡📰!#Tourism #hazard #kompany #coronavirus…,Auxipress,Belgium,"[(today, DATE), (Tourism, PERSON), ]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",Belgium
1,None,"Iran: #Coronavirus Update, Over 44,800 Deaths, May 27, 2020, 6:00 PM CEST - NCRI #HumanRightsV…",davedaram,,"[(Iran, GPE), (Over 44,800, CARDINAL), (May 27, 2020, DATE), (6:00 PM, TIME), (CEST, PERSON), (HumanRightsV, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",
2,None,#Coronavirus has not stopped Modi-Shah's police to target Muslim students for participating in anti-citizenship act protest!!…,israrul8,,"[(Coronavirus, PERSON), (Modi-Shah's, ORG), (Muslim, NORP)]","{'neg': 0.0, 'neu': 0.87, 'pos': 0.13, 'compound': 0.3071}",
3,None,Does anyone still read the #NYT ? 🤣🤣“The New York Times admitted this week the doctor didn’t die and didn’t have coronavirus…,Aliciastarr001,California (Bay Area),"[(🤣🤣, PERSON), (The New York Times, ORG), (this week, DATE)]","{'neg': 0.16, 'neu': 0.782, 'pos': 0.058, 'compound': -0.5423}",USA
4,None,"Those on the front lines of the COVID-19 fight need quick, easy access to real-time critical data. That’s what APIs (and AP…",sounakpal,India,"[(AP, ORG)]","{'neg': 0.183, 'neu': 0.709, 'pos': 0.108, 'compound': -0.25}",India
5,None,"""Freedom will become the rule again and prohibition will be the exception,"" «The end of #confinement will allow us…",domwabs,"Lyon, France",[],"{'neg': 0.0, 'neu': 0.736, 'pos': 0.264, 'compound': 0.7269}",France
6,None,Two countries united efforts to kill the #CoronaVirus in between them.PS. Sharp weapons used,Vas_Ah,"Jammu, Jammu And Kashmir","[(Two, CARDINAL), (CoronaVirus, PERSON)]","{'neg': 0.355, 'neu': 0.514, 'pos': 0.131, 'compound': -0.7003}",Jammu
7,None,Keeping the spirit of generosity alive during #COVID19 pandemic. #relief programme at Thaiba Garden and near by villages.…,rafeek_rapi,,"[(Thaiba Garden, FAC)]","{'neg': 0.0, 'neu': 0.664, 'pos': 0.336, 'compound': 0.765}",
8,None,"""Government rejected radical lockdown of England's care homes"" #coronavirus Is anyone surprised? This govt is so cl…",buddah_uk,Not in Surrey,"[(England, GPE)]","{'neg': 0.147, 'neu': 0.625, 'pos': 0.228, 'compound': 0.2023}",Canada
9,None,The smoking gun of govt care home policy?Govt advice from 2 April.“Some of these patients may have #COVOD19. All ca…,tonyevansace,,"[(2 April, DATE), (COVOD19, MONEY)]","{'neg': 0.106, 'neu': 0.752, 'pos': 0.142, 'compound': 0.2023}",


In [0]:
tweet_df.to_csv('tweets.csv')